In [102]:
from selenium.webdriver.firefox.options import Options
from selenium.webdriver import DesiredCapabilities
from selenium import webdriver
from selenium.webdriver.firefox.service import Service

from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException,TimeoutException
from selenium.webdriver.common.by import By
from explicit import waiter, XPATH
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.alert import Alert

import random
import time
import pandas as pd
from bs4 import BeautifulSoup
import json
import time
import re
from urllib.parse import urlparse
import configparser
import logging

In [103]:
def read_settings():
    config = configparser.ConfigParser()
    config.read('config.ini')

    search_engine = config.get("CTR SETTINGS","SEARCH_ENGINE")
    target_url = config.get("CTR SETTINGS","TARGET_URL")
    keywords = config.get("CTR SETTINGS","KEYWORDS").split('\n')
    parsed_url = urlparse(target_url)
    target_domain_name = parsed_url.netloc
    
    
    return search_engine,target_url,keywords,target_domain_name

In [104]:
search_engine,target_url,keywords,target_domain_name = read_settings()

In [105]:
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))

[WDM] - Downloading: 19.2kB [00:00, 19.2MB/s]                                             


In [106]:
driver.get('https://www.bing.com/')
driver.refresh()

WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.NAME, "q"))
    )
search_box = driver.find_element(By.NAME, "q")
keyword = random.choice(keywords)

for letter in keyword:
    search_box.send_keys(letter)
    time.sleep(random.randint(0,1000)/1000)

time.sleep(2)

search_box.send_keys(Keys.RETURN)

time.sleep(5)

In [107]:
flag = True;counter=1
logging.basicConfig(level=logging.DEBUG, filename='logger.log', filemode='w',
                    format='%(asctime)s - %(levelname)s - %(message)s')
logging.info(f"\n\n\n ========= {keyword} ========= \n")
while flag:
    if counter>10:
        break
    logging.info(f"Currently in --> Page-{counter}")
    print(f"Currently in --> Page-{counter}")
    try:
        html_source = driver.page_source
        sourcedata = html_source.encode('utf-8')
        soup = BeautifulSoup( sourcedata , 'html.parser')
        
        search_results = soup.find_all("li", {"class": "b_algo"})
        for result in search_results:
            hyperlink = result.find("cite").text
            title = result.find("h2").text
            logging.info(f"{counter}.{hyperlink} \n")
        
            print(hyperlink,"-->",title)
            if target_domain_name in hyperlink:
                target_title = title
                print(f"====== Found The Link ======= at index - {counter}")
                flag=False
                break
        if flag:
            next_button = driver.find_element(By.XPATH, "//a[@title='Next page']")
            
            desired_y = (next_button.size['height'] / 2) + next_button.location['y']
            window_h = driver.execute_script('return window.innerHeight')
            window_y = driver.execute_script('return window.pageYOffset')
            current_y = (window_h / 2) + window_y
            scroll_y_by = desired_y - current_y

            for i in range(1, int(scroll_y_by), 1):
                driver.execute_script("window.scrollTo(0, {});".format(i))
            time.sleep(3)
            next_button.click()
            counter+=1
            time.sleep(5)
    except Exception as e:
        print(e)
        break
        

Currently in --> Page-1
https://www.streamscheme.com/how-to-block-ad… --> How To Block Ads On Twitch In 2023 - StreamScheme
https://cybernews.com/best-ad-blockers/ad-blocker-for-twitch --> Best ad blockers for Twitch in 2023 - Cybernews
====== Found The Link ======= at index - 1


In [113]:
if not flag:
    h2_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT,target_title))
    )
    desired_y = (h2_element.size['height'] / 2) + h2_element.location['y']
    window_h = driver.execute_script('return window.innerHeight')
    window_y = driver.execute_script('return window.pageYOffset')
    current_y = (window_h / 2) + window_y
    scroll_y_by = desired_y - current_y
    
    for i in range(1, int(scroll_y_by), 1):
        driver.execute_script("window.scrollTo(0, {});".format(i))
    time.sleep(5)
    h2_element.click()
else:
    driver.get(target_url)